In [56]:
#Importing required headers.
import numpy as np
import pandas as pd
import operator

#Reading Dataset
df = pd.read_csv('Data.csv')
df.columns = ["batsman", "bowler", "runs", "wicket"]

#Getting all the unique batsmen and bowlers
batsmen = df[df.columns[0]]
bowlers = df[df.columns[1]]
bowlers_unique = list(set(bowlers))
batsmen_unique = list(set(batsmen))

#Ordered List representing wickets. 
out_or_not = df[df.columns[3]]

#To compute Batsmen Vulnerability
def Compute1(x):
    dic = {}
    bat = x
    for i in range(0,len(bowlers_unique)):
        dic[bowlers_unique[i]] = 0
    z = df[df["batsman"] == x]
    z = z[z["wicket"] == "OUT"]
    z = list(z[z.columns[1]])
    for i in range(0,len(z)):
        dic[z[i]] = dic[z[i]]+1
    y = max(dic.items(), key=operator.itemgetter(1)) 
    return [y[0], y[1]]

#To compute Bowler Vulnerability
def Compute2(x):
    dic = {}
    bowler = x
    for i in range(0,len(batsmen_unique)):
        dic[batsmen_unique[i]] = 0
    z = df[df["bowler"] == x]
    b = list(z[z.columns[0]])
    r = list(z[z.columns[2]])
    for i in range(0,len(b)):
        dic[b[i]] = dic[b[i]] + r[i]
    y = max(dic.items(), key=operator.itemgetter(1)) 
    return [y[0], y[1]]

In [57]:
#Batsmen Vulnerability

%%time
batsmanV = {}
for i in batsmen_unique:
    z = Compute1(i)
    batsmanV[i] = z   

x = pd.DataFrame.from_dict(batsmanV, orient='index')
x = x.reset_index()
x.columns = ["Batsman", "Bowler", "No. times OUT"]
x.index.name = "Row No."
print(x)
print()
print(x[x['No. times OUT']==x['No. times OUT'].max()])
print()

SyntaxError: invalid syntax (<ipython-input-57-6f029c2064b1>, line 3)

In [55]:
#Bowler Vulnerability

%%time
bowlerV = {}
for i in bowlers_unique:
    z = Compute2(i)
    bowlerV[i] = z
    
x = pd.DataFrame.from_dict(bowlerV, orient='index')
x = x.reset_index()
x.columns = ["Bowler", "Batsman", "Runs Given"]
x.index.name = "Row No."
print(x)
print()
print(x[x['Runs Given']==x['Runs Given'].max()])
print()

SyntaxError: invalid syntax (<ipython-input-55-4e0efd1e8df3>, line 3)

In [50]:
#Calculation of player to player probabilities.

def CalculateProbability(batsman, bowler):
    #DataStructure used is a array of length 8.
    #Format of array is: Batsman, Bowler, p(0's), p(1's), p(2's), p(3's), p(4's), p(6's), p(Outs), Total Balls.
    array = ["x", "y", 0, 0, 0, 0, 0, 0, 0, 0]
        
    #Filtering the dataset for the particular batsman and bowler part.
    temp = df[df["batsman"] == batsman]
    temp = temp[temp["bowler"] == bowler]
    
    zeros = np.count_nonzero(temp.runs == 0)
    ones = np.count_nonzero(temp.runs == 1)
    twos = np.count_nonzero(temp.runs == 2)
    threes = np.count_nonzero(temp.runs == 3)
    fours = np.count_nonzero(temp.runs == 4)
    sixes = np.count_nonzero(temp.runs == 6)
    outs = np.count_nonzero(temp.wicket == 'OUT')
    totBalls = len(temp)
    
    array[0] = batsman
    array[1] = bowler
    
    if(totBalls != 0):
        array[2] = zeros/totBalls
        array[3] = ones/totBalls
        array[4] = twos/totBalls
        array[5] = threes/totBalls
        array[6] = fours/totBalls
        array[7] = sixes/totBalls
        array[8] = outs/totBalls
        array[9] = totBalls

    return array

In [51]:
CalculateProbability("F du Plessis", "Z Khan")

['F du Plessis',
 'Z Khan',
 0.35714285714285715,
 0.2857142857142857,
 0.07142857142857142,
 0.0,
 0.2857142857142857,
 0.0,
 0.0,
 14]

In [58]:
#Calcualting the Probabilities for all players:

x = list()

for i in batsmen_unique:
    for j in bowlers_unique:
        x.append(CalculateProbability(i, j))
        
dataframe = pd.DataFrame(x)

In [71]:
dataframe = dataframe.drop(['index'],axis = 1)
dataframe = dataframe.drop(['level_0'],axis = 1)

In [76]:
dataframe.columns = ["Batsman", "Bowler", "p(0's)", "p(1's)", "p(2's)", "p(3's)", "p(4's)", "p(6's)", "p(Out)", "TotalBalls"]
dataframe.index.name = "RowNo."

In [77]:
dataframe

Batsman           Bowler    p(0's)    p(1's)    p(2's)  p(3's)  \
RowNo.                                                                          
0       A Ashish Reddy   A Ashish Reddy  0.000000  0.000000  0.000000   0.000   
1       A Ashish Reddy   AD Mascarenhas  0.000000  0.000000  0.000000   0.000   
2       A Ashish Reddy        SS Sarkar  0.000000  0.000000  0.000000   0.000   
3       A Ashish Reddy         R Bhatia  0.000000  0.000000  0.000000   0.000   
4       A Ashish Reddy    Kuldeep Yadav  0.000000  0.000000  0.000000   0.000   
5       A Ashish Reddy         AJ Finch  0.000000  0.000000  0.000000   0.000   
6       A Ashish Reddy       KA Pollard  0.333333  0.333333  0.333333   0.000   
7       A Ashish Reddy         A Kumble  0.000000  0.000000  0.000000   0.000   
8       A Ashish Reddy         PV Tambe  0.250000  0.750000  0.000000   0.000   
9       A Ashish Reddy     F du Plessis  0.000000  0.000000  0.000000   0.000   
10      A Ashish Reddy          A Singh  0.000000  0.000000  0.000000   0.000   
11      A Ashish Reddy    SJ Srivastava  0.000000  0.000000  0.000000   0.000   
12      A Ashish Reddy    Anureet Singh  0.000000  1.000000  0.000000   0.000   
13      A Ashish Reddy          S Gopal  0.000000  0.000000  0.000000   0.000   
14      A Ashish Reddy      Imran Tahir  0.000000  1.000000  0.000000   0.000   
15      A Ashish Reddy         R Sharma  0.000000  0.000000  0.000000   0.000   
16      A Ashish Reddy    CR Brathwaite  0.000000  0.000000  0.000000   0.000   
17      A Ashish Reddy      DJ Thornely  0.000000  0.000000  0.000000   0.000   
18      A Ashish Reddy        S Aravind  0.000000  0.000000  0.000000   0.000   
19      A Ashish Reddy        V Shankar  0.000000  0.000000  0.000000   0.000   
20      A Ashish Reddy        HH Pandya  0.000000  0.000000  0.000000   0.000   
21      A Ashish Reddy     TP Sudhindra  0.000000  0.000000  0.000000   0.000   
22      A Ashish Reddy        MB Parmar  0.000000  0.000000  0.000000   0.000   
23      A Ashish Reddy       MP Stoinis  0.000000  0.000000  0.000000   0.000   
24      A Ashish Reddy    Swapnil Singh  0.000000  0.000000  0.000000   0.000   
25      A Ashish Reddy  JJ van der Wath  0.000000  0.000000  0.000000   0.000   
26      A Ashish Reddy       RR Bhatkal  0.000000  0.000000  0.000000   0.000   
27      A Ashish Reddy         DW Steyn  0.000000  0.000000  0.000000   0.000   
28      A Ashish Reddy           K Goel  0.000000  0.000000  0.000000   0.000   
29      A Ashish Reddy          S Ladda  0.000000  0.000000  0.000000   0.000   
...                ...              ...       ...       ...       ...     ...   
164086       MK Tiwary         S Sriram  0.000000  0.000000  0.000000   0.000   
164087       MK Tiwary       Avesh Khan  0.000000  0.000000  0.000000   0.000   
164088       MK Tiwary       AUK Pathan  0.000000  0.000000  0.000000   0.000   
164089       MK Tiwary       JD Unadkat  0.300000  0.400000  0.100000   0.000   
164090       MK Tiwary    DP Vijaykumar  0.000000  0.000000  0.000000   0.000   
164091       MK Tiwary  C de Grandhomme  0.000000  0.000000  0.000000   0.000   
164092       MK Tiwary      S Sreesanth  0.000000  0.500000  0.500000   0.000   
164093       MK Tiwary         AN Ahmed  0.500000  0.000000  0.000000   0.000   
164094       MK Tiwary       A Chandila  0.000000  0.000000  0.000000   0.000   
164095       MK Tiwary     Shoaib Malik  0.000000  0.000000  0.000000   0.000   
164096       MK Tiwary  Karanveer Singh  0.166667  0.666667  0.166667   0.000   
164097       MK Tiwary         MG Neser  0.000000  0.000000  0.000000   0.000   
164098       MK Tiwary    Sohail Tanvir  0.000000  0.000000  0.000000   0.000   
164099       MK Tiwary       SM Harwood  0.000000  0.000000  0.000000   0.000   
164100       MK Tiwary         AF Milne  0.285714  0.357143  0.142857   0.000   
164101       MK Tiwary         FY Fazal  0.000000  0.000000  0.000000   0.000   
164102       MK Tiwary        JA Morkel  0.277

In [78]:
dataframe.to_csv("./Probabilities.csv")